In [1]:
# import os

# os.chdir('src/poc')

In [2]:
import tomllib
import numpy as np
from pprint import pprint
import librosa
import requests
from pydub import AudioSegment
from io import BytesIO
from SpotifyTrack import SpotifyTrack
from poc_test import estimate_tempo
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = None
sp = None

with open('cfg.toml', 'rb') as cfg:
    sptfy_cfg = tomllib.load(cfg)['spotify']
    client_id = sptfy_cfg['client_id']
    client_secret = sptfy_cfg['client_secret']
    auth_manager = SpotifyClientCredentials(client_id=client_id,
                                            client_secret=client_secret)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
# The Weeknd's "After Hours" 4yP0hdKOZPNshxUOjY0cZj
(album_name, album_uri) = ('harvest','spotify:album:7hIjNhXoPqyARBpaKpS3nk')
# (album_name, album_uri) = ('sdp_my_eyes', 'spotify:album:2nhThWvrfDbjXvRfuwlsCE')
# (album_name, album_uri) = ('avril_lavigne', 'spotify:album:3zXjR3y2dUWklKmmp6lEhy')
# (album_name, album_uri) = ('travy_patty', 'spotify:album:18NOKLkZETa4sWwLMIm0UZ')

tracks_raw = sp.album_tracks(album_id=album_uri.split(':')[-1], market='CA')

tracks = [SpotifyTrack(**(tracks_raw['items'][i])) for i in range(len(tracks_raw['items']))]

In [4]:
tracks_raw

{'href': 'https://api.spotify.com/v1/albums/7hIjNhXoPqyARBpaKpS3nk/tracks?offset=0&limit=50&market=CA',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2CtiNJi0zB35RHYXViVjKG'},
     'href': 'https://api.spotify.com/v1/artists/2CtiNJi0zB35RHYXViVjKG',
     'id': '2CtiNJi0zB35RHYXViVjKG',
     'name': 'BALTHVS',
     'type': 'artist',
     'uri': 'spotify:artist:2CtiNJi0zB35RHYXViVjKG'}],
   'disc_number': 1,
   'duration_ms': 235000,
   'explicit': False,
   'external_urls': {'spotify': 'https://open.spotify.com/track/2Rlw1EslDSKSODPst1GhOY'},
   'href': 'https://api.spotify.com/v1/tracks/2Rlw1EslDSKSODPst1GhOY',
   'id': '2Rlw1EslDSKSODPst1GhOY',
   'is_playable': True,
   'name': 'Sun Colored Eyes',
   'preview_url': 'https://p.scdn.co/mp3-preview/edd53ab647b8140a72f5284d8a0707aac6d37321?cid=970a76441550417c8d7779d78038bc31',
   'track_number': 1,
   'type': 'track',
   'uri': 'spotify:track:2Rlw1EslDSKSODPst1GhOY',
   'is_local': False},
  {'ar

In [5]:
tracks

In [6]:
def save_wavs(tracks):
    for track in tracks:
        r = requests.get(track.preview_url)
        audio = AudioSegment.from_mp3(BytesIO(r.content))
        wav_path = f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav'
        audio.export(wav_path, format='wav')

In [7]:
def get_tempo_matt(wav_path):
    signal, fs = librosa.load(wav_path, sr=11025, mono=True)
    signal = signal / np.max(np.abs(signal))
    window_size = 1024
    hop_size = 512
    fs = 11025
    tempo, _, _, _ = estimate_tempo(signal, fs, window_size, hop_size)
    return tempo

In [8]:
acc_tempos = [sp.audio_features(harvest_track.id)[0].get('tempo') for harvest_track in tracks]

In [ ]:
# save_wavs(tracks)

In [10]:
matt_tempos = [get_tempo_matt(f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav').item() for track in tracks]

In [11]:
def get_tempo_matt(wav_path):
    signal, fs = librosa.load(wav_path, sr=11025, mono=True)
    signal = signal / np.max(np.abs(signal))

    window_size = 1024
    hop_size = 512
    fs = 11025

    tempo, _, _, _ = estimate_tempo(signal, fs, window_size, hop_size)

    return tempo

In [13]:
pd.DataFrame(zip(matt_tempos, acc_tempos),
             index=(t.name for t in tracks),
             columns=['OurAlgoTempo', 'SpotifyTempo'])

,OurAlgoTempo,SpotifyTempo
Sun Colored Eyes,46.142578,91.566
Asha,43.066406,86.406
Anouk,99.384014,149.347
Like Coconut Water,64.599609,96.949
Lovin',143.554688,142.667
Aguacero,184.570312,98.371
Mango Season,184.570312,87.014
Sun & Moon,107.666016,105.805
Venus Flytrap,215.332031,108.558
Aio,143.554688,110.685
